---
<div style="text-align: center;" >
    <h1>
        <b>データサイエンスプログラミング演習</b>
    </h1>
    <h3>
        第11,12回授業ノートの続き
    </h3>
</div>
<div style="text-align: right">
    19220001<br/>
    中村亮太
</div>

---

# 実践データ分析　ケース2
## 汚いデータの扱い
### ECサイトの売上げデータ
ある小売店の売上履歴と顧客台帳データを用いて、データ分析の素地となる「データの加工」を習得する。実際の現場データは手入力のExcel等、決して綺麗なデータではないことが多い。データの揺れや整合性の担保など、汚いデータを取り扱うデータ加工を主体に進める。
- ステップ1：データの読込
- ステップ2：データの揺れをみる
- ステップ3：商品名の揺れを補正する
- ステップ4：欠損値を補完する
- ステップ5：顧客名の揺れも補正する
- ステップ6：日付の揺れも補正する
- ステップ7：顧客名を主軸に2つのデータを結合（ジョイン）する
- ステップ8：クレンジングしたデータを俯瞰する
- ステップ9：データ集計する

#### ステップ1 データの読込
- 売上げデータ（sales.csv）
- 顧客情報（customer_info.xlsx）

### 売上げデータ(sales.csv）を読み込む

In [102]:
import pandas as pd
sales_data = pd.read_csv('sales.csv')

#先頭部分だけを表示して様子を見る
sales_data.head()
#NaNの意味とは、「Not a Number」 、Noneとは別物

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


### 顧客情報(customer_info.xlsx）を読み込む

In [103]:
customer_data = pd.read_excel('customer_info.xlsx')
#先頭部分だけを表示して様子を見る
customer_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


---

#### ステップ2 データの揺れをみる

### 各種データ内に存在するデータの揺れと欠損値の有無を確認する

In [104]:
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 5)

In [105]:
#sales_data全体を眺める
sales_data

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
...,...,...,...,...
2997,2019-07-14 12:56:49,商品H,NaN,芦田博之
2998,2019-07-21 00:31:36,商品D,400.0,石田郁恵


In [106]:
#customer_data全体を眺める
customer_data

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
...,...,...,...,...,...
198,大西 隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019/04/19
199,福井 美希,ふくい みき,D市,fukui_miki1@example.com,2019/04/23


In [107]:
#sales_dataの'item_name'列だけを眺める
sales_data["item_name"]

0         商品A
1       商 品 S
        ...  
2997      商品H
2998      商品D
Name: item_name, Length: 2999, dtype: object

In [108]:
#sales_dataの'item_price'列だけを眺める
sales_data["item_price"]

0       100.0
1         NaN
        ...  
2997      NaN
2998    400.0
Name: item_price, Length: 2999, dtype: float64

In [109]:
#customer_dataの'登録日'列だけを眺める
customer_data["登録日"]

0      2018/01/04
1           42782
          ...    
198    2019/04/19
199    2019/04/23
Name: 登録日, Length: 200, dtype: object

### データに揺れがあるまま集計してみる
商品Sと商品s など、本来同じ商品が別の商品として集計されている。データの揺れがあるため、本来26個の商品が99個に増えてしまっている。

In [110]:
sales_data['purchase_date'] = pd.to_datetime(sales_data['purchase_date'])

sales_data['purchase_month'] = sales_data['purchase_date'].dt.strftime('%Y/%m')

res = sales_data.pivot_table(
    index='purchase_month', 
    columns='item_name', 
    aggfunc='size', 
    fill_value=0 #??????
)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019/01,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
2019/02,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/06,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2019/07,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


#### データの揺れが残ったままの集計や分析は全く意味のない結果になってしまう。やはりデータ加工が分析の前処理として重要である。


---

#### ステップ3 商品名の揺れを補正する

In [111]:
#まずは現状の確認（これも重要）
#売上げ履歴のitem_nameの重複を除外したユニークなデータ件数を出力する
#商品Sや商品s などがあるので、データ件数は99個になってしまう
print(len(pd.unique((sales_data['item_name']))))

99


In [112]:
#大文字にそろえる　小文字→大文字に変換
sales_data['item_name'] = sales_data['item_name'].str.upper()

#全角スペースを除去する
sales_data['item_name'] = sales_data['item_name'].str.replace("　","")

#半角スペースを除去する
sales_data['item_name'] = sales_data['item_name'].str.replace(" ","")

#item_nameを名前順にソートして見やすくする
sales_data.sort_values(by=['item_name'], ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,2019/06
1748,2019-05-19 20:22:22,商品A,100.0,松川綾女,2019/05
...,...,...,...,...,...
1361,2019-05-28 13:45:32,商品Y,2500.0,大崎ヒカル,2019/05
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,2019/02


In [113]:
print(pd.unique(sales_data['item_name']))
print(len(pd.unique(sales_data['item_name'])))
#99→26に整理できる

['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']
26


---

#### ステップ4 欠損値を補完する

In [114]:
sales_data.isnull().any(axis=0)
#isnull()で欠損値の有無を確認できる axis=0・・・列単位でチェック　axis=1・・・行単位でチェック

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

item_priceがTrueになっているので、この項目に欠損値が含まれていることを確認できる

In [115]:
#NaNが含まれる行だけを表示する
sales_data[sales_data.isnull().any(axis=1)]

,purchase_date,item_name,item_price,customer_name,purchase_month
1,2019-07-13 13:05:29,商品S,NaN,浅田賢二,2019/07
2,2019-05-11 19:42:07,商品A,NaN,南部慶二,2019/05
...,...,...,...,...,...
2996,2019-03-29 11:14:05,商品Q,NaN,尾形小雁,2019/03
2997,2019-07-14 12:56:49,商品H,NaN,芦田博之,2019/07


欠損値を補完する<br>
今回のケースでは、欠損値は正しく記載された他の行から同じ商品の単価を調べ補完できる（集計期間中の商品単価の変動はないという前提条件）

In [116]:
#欠損値のある箇所を特定する
#flg_is_null変数には、欠損値がある行はTrue、欠損値がない行はFalseになったデータが格納される
flg_is_null = sales_data['item_price'].isnull()
flg_is_null

0       False
1        True
        ...  
2997     True
2998    False
Name: item_price, Length: 2999, dtype: bool

In [117]:
sales_data.loc[(flg_is_null==False)&(sales_data["item_name"]=="商品A"),'item_price']
#sales_data.loc[~flg_is_null&(sales_data["item_name"]=="商品A"),'item_price']

0       100.0
9       100.0
        ...  
2951    100.0
2973    100.0
Name: item_price, Length: 117, dtype: float64

In [118]:
#flg_is_nullに格納されている「item_name」の列データから取り出した99個のユニークな商品名を対象にfor文をまわす
#unigue（）は抽出した商品名の重複をなくすために用いられる

for trg in list(sales_data.loc[flg_is_null, 'item_name'].unique()):
    
    #trgには欠損値がある商品名が入ってくる。
    #trgを用いて、同じ商品で金額が正しく記載されている行を.locで探し、その金額を取得
    #~flg_is_nullの「〜」は否定演算子 　flg_is_null == Falseと同じ意味
    #flg_is_null　は flg_is_null == Trueと同じ意味
    price = sales_data.loc[
        ~flg_is_null & (sales_data['item_name'] == trg), 
        'item_price'
    ].max()
    
    sales_data['item_price'].loc[
        flg_is_null & (sales_data['item_name']==trg)
    ] = price
    
sales_data

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,2019/06
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,2019/07
...,...,...,...,...,...
2997,2019-07-14 12:56:49,商品H,800.0,芦田博之,2019/07
2998,2019-07-21 00:31:36,商品D,400.0,石田郁恵,2019/07


In [119]:
#欠損値が補完されたかどうかをチェック
sales_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

### データ検算

In [120]:
#商品Aの価格が正しく補完されているかチェック
#すべての商品Aを確認し最大値を算出する
location     = sales_data['item_name']=='商品A'
sales_data_A = sales_data.loc[location]
max_value    = sales_data_A['item_price'].max()
max_value

100.0

In [121]:
#商品Aの最小値を算出する
min_value    = sales_data_A['item_price'].min()
min_value

100.0

In [122]:
#sales_data.loc[sales_data['item_name']=='商品A']['item_price'].max()

sales_data.loc[
    sales_data['item_name']=='商品A'
][
    'item_price'
].max()

100.0

In [123]:
#商品A〜Zの価格の最大値と最小値を算出、正しく補完されていれば最大値と最小値が等しいはず
for trg in list(sales_data['item_name'].sort_values().unique()):
    print(trg 
          + 'の最大額：' 
          + str(
              sales_data.loc[
                  sales_data["item_name"]==trg
              ][
                  "item_price"
              ].max()
          )
         )
    print(trg
          + 'の最小額：' 
          + str(
              sales_data.loc[
                  sales_data["item_name"]==trg
              ][
                  "item_price"
              ].min(skipna=False)
          )
         )
         #skipnaはNaNデータを無視するかどうかを設定できる。FalseにするとNaNがあったときNaNと出力される

商品Aの最大額：100.0
商品Aの最小額：100.0
商品Bの最大額：200.0
商品Bの最小額：200.0
商品Cの最大額：300.0
商品Cの最小額：300.0
商品Dの最大額：400.0
商品Dの最小額：400.0
商品Eの最大額：500.0
商品Eの最小額：500.0
商品Fの最大額：600.0
商品Fの最小額：600.0
商品Gの最大額：700.0
商品Gの最小額：700.0
商品Hの最大額：800.0
商品Hの最小額：800.0
商品Iの最大額：900.0
商品Iの最小額：900.0
商品Jの最大額：1000.0
商品Jの最小額：1000.0
商品Kの最大額：1100.0
商品Kの最小額：1100.0
商品Lの最大額：1200.0
商品Lの最小額：1200.0
商品Mの最大額：1300.0
商品Mの最小額：1300.0
商品Nの最大額：1400.0
商品Nの最小額：1400.0
商品Oの最大額：1500.0
商品Oの最小額：1500.0
商品Pの最大額：1600.0
商品Pの最小額：1600.0
商品Qの最大額：1700.0
商品Qの最小額：1700.0
商品Rの最大額：1800.0
商品Rの最小額：1800.0
商品Sの最大額：1900.0
商品Sの最小額：1900.0
商品Tの最大額：2000.0
商品Tの最小額：2000.0
商品Uの最大額：2100.0
商品Uの最小額：2100.0
商品Vの最大額：2200.0
商品Vの最小額：2200.0
商品Wの最大額：2300.0
商品Wの最小額：2300.0
商品Xの最大額：2400.0
商品Xの最小額：2400.0
商品Yの最大額：2500.0
商品Yの最小額：2500.0
商品Zの最大額：2600.0
商品Zの最小額：2600.0


---

#### ステップ5 顧客名の揺れも補正する

In [124]:
sales_data['customer_name'].head()

0    深井菜々美
1     浅田賢二
2     南部慶二
3     麻生莉緒
4     平田鉄二
Name: customer_name, dtype: object

In [125]:
customer_data['顧客名'].head()

0     須賀ひとみ
1    岡田　 敏也
2      芳賀 希
3     荻野  愛
4     栗田 憲一
Name: 顧客名, dtype: object

**このままでは、売上げ履歴と顧客台帳を結合してもだたしく結合できない**

In [132]:
#半角・全角スペースを除去する
customer_data['顧客名'] = customer_data['顧客名'].str.replace("　","")
customer_data['顧客名'] = customer_data['顧客名'].str.replace(" ","")
customer_data['顧客名'].head()

0    須賀ひとみ
1     岡田敏也
2      芳賀希
3      荻野愛
4     栗田憲一
Name: 顧客名, dtype: object

**実際のデータの中には名前の誤変換などの複雑な揺れが存在することも多々ある。その場合はプログラムで補正することができないため、現場の運用スタッフにヒアリングし、地道に名寄せ作業を行う必要がある。**

---

#### ステップ6 日付の揺れも補正する

In [133]:
customer_data['登録日'].head()

0    2018/01/04
1         42782
2    2018/01/07
3         42872
4         43127
Name: 登録日, dtype: object

**登録日の列には「42782」のように日付でない数字がいくつか見られる。Excelデータに格納されたデータに書式が違うデータが混在することはよくある。**

In [136]:
#日付を統一フォーマットに補正する前に状況把握
#str.isdigit()を用いて、顧客台帳の登録日が数値かどうかをチェックする　True・・・数値　False・・・数値ではない
flg_is_serial = customer_data['登録日'].astype("str").str.isdigit()
flg_is_serial

0      False
1       True
       ...  
198    False
199    False
Name: 登録日, Length: 200, dtype: bool

In [137]:
#登録日が数値データになっている件数を確認する
flg_is_serial.sum()

22

In [138]:
#日付を統一フォーマットに補正する
fromSerial = pd.to_timedelta( #pd.to_timedelta()関数を用いて、数値から日付に変換する
    customer_data.loc[flg_is_serial, '登録日'], #locを用いてflg_is_serialの条件でデータを抽出
    unit='d') + pd.to_datetime('1900/01/01') #フォーマットを指定する
fromSerial

1     2017-02-18
3     2017-05-19
         ...    
186   2018-07-15
192   2018-06-10
Name: 登録日, Length: 22, dtype: datetime64[ns]

In [139]:
#もともと日付だったデータも、書式統一のために処理する
fromString = pd.to_datetime(customer_data.loc[~flg_is_serial, '登録日'])
#~flg_is_serialは(flg_is_serial==False)と同じ
fromString

0     2018-01-04
2     2018-01-07
         ...    
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 178, dtype: datetime64[ns]

In [140]:
#数値から日付に補正したデータと、書式を変更したデータを結合しcustomer_data['登録日']データを更新する
customer_data['登録日'] = pd.concat([fromSerial, fromString])
customer_data

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18
...,...,...,...,...,...
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19
199,福井美希,ふくい みき,D市,fukui_miki1@example.com,2019-04-23


In [141]:
#データ検算 数値項目が残っていないかチェックする
flg_is_serial = customer_data['登録日'].astype("str").str.isdigit()
flg_is_serial.sum()

0

**数値データ件数は「0件」となり、すべての数値データが日付に補正されたことが確認できた**

In [142]:
#登録日から登録付を算出し、集計する
customer_data['登録年月'] = customer_data['登録日'].dt.strftime("%Y/%m")
customer_data['登録年月']

0      2018/01
1      2017/02
        ...   
198    2019/04
199    2019/04
Name: 登録年月, Length: 200, dtype: object

In [143]:
rslt = customer_data.groupby('登録年月').count()['顧客名']
print(rslt)
print(len(customer_data))

登録年月
2017/01    15
2017/02    11
           ..
2018/07    17
2019/04     2
Name: 顧客名, Length: 15, dtype: int64
200


---

#### ステップ7 顧客名を主軸に2つのデータを結合（ジョイン）する
売上げ履歴と顧客台帳を結合し、集計のベースとなるデータを作成する

In [144]:
sales_data.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,2019/06
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,2019/07
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,2019/05
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,2019/02
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,2019/04


In [145]:
customer_data.head()

,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,2018/01
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18,2017/02
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07,2018/01
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19,2017/05
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29,2018/01


**共通する列名がない。**

In [146]:
join_data = pd.merge(
    sales_data,
    customer_data,
    left_on="customer_name",
    right_on="顧客名",
    how='left') #sales_dataを主としてcustomer_dataを結合する
join_data.head()

,purchase_date,item_name,item_price,customer_name,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,2019/06,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,2017/01
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,2019/07,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,2018/04
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,2019/05,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,2018/06
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,2019/02,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,2018/07
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,2019/04,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,2017/06


In [149]:
join_data = join_data.drop('customer_name', axis=1)
join_data.head()

,purchase_date,item_name,item_price,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,2019/06,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,2017/01
1,2019-07-13 13:05:29,商品S,1900.0,2019/07,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,2018/04
2,2019-05-11 19:42:07,商品A,100.0,2019/05,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,2018/06
3,2019-02-12 23:40:45,商品Z,2600.0,2019/02,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,2018/07
4,2019-04-22 03:09:35,商品A,100.0,2019/04,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,2017/06


---

#### ステップ8 クレンジングしたデータを俯瞰する
きれいになったデータをファイル出力（ダンプ）して、分析をする際は出力ファイルから読込分析を行うことで、クレンジングのやり直しを省略する

In [148]:
#ダンプする前に列を並び替え、より分かりやすくする。例えば、購入日付情報を隣り合わせにする
dump_data = join_data[
    [
        'purchase_date', 
        'purchase_month', 
        'item_name', 
        'item_price', 
        '顧客名', 
        'かな', 
        '地域', 
        'メールアドレス', 
        '登録日'
    ]
]
dump_data.head()

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,2019/06,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,2019/07,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,2019/05,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,2019/02,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,2019/04,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07


In [150]:
#csvファイル「dump_data.csv」に出力する
dump_data.to_csv('dump_data.csv', index=False)

---

#### ステップ9 データを集計する

In [151]:
#さきほどダンプした「dump_data.csv」を読み込んでデータ集計の準備をする
import_data = pd.read_csv('dump_data.csv')
import_data.head()

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,2019/06,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26 00:00:00
1,2019-07-13 13:05:29,2019/07,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07 00:00:00
2,2019-05-11 19:42:07,2019/05,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19 00:00:00
3,2019-02-12 23:40:45,2019/02,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22 00:00:00
4,2019-04-22 03:09:35,2019/04,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07 00:00:00


In [152]:
#purchase_monthを縦軸に、商品ごとの集計結果を表示する
byItem = import_data.pivot_table(
    index='purchase_month', 
    columns='item_name', 
    aggfunc='size', 
    fill_value=0
)
byItem

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019/01,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
2019/02,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/06,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
2019/07,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [153]:
#purchase_monthを縦軸に、商品ごとの売上金額を集計する
byPrice = import_data.pivot_table(
    index='purchase_month', 
    columns='item_name', 
    values='item_price', 
    aggfunc='sum', 
    fill_value=0
)
byPrice

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019/01,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
2019/02,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/06,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
2019/07,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [158]:
#purchase_monthを縦軸に、各顧客の購入数を集計する
byCustomer = import_data.pivot_table(
    index='purchase_month',
    columns='item_name',
    aggfunc='size',
    fill_value=0
)
byCustomer

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019/01,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
2019/02,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/06,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
2019/07,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [159]:
#purchase_monthを縦軸に、地域ごとの購入数を集計する
byRegion = import_data.pivot_table(
    index='purchase_month', 
    columns='item_name', 
    aggfunc='size', 
    fill_value=0
)
byRegion

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019/01,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
2019/02,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/06,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
2019/07,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [157]:
#customer_dataと主としてsales_dataを結合し、集計期間で商品を購入していないユーザがいないかチェックする準備を行う
away_data = pd.merge( 
    customer_data, 
    sales_data,
    left_on='顧客名', 
    right_on='customer_name', 
)
away_data.head()

,顧客名,かな,地域,メールアドレス,登録日,登録年月,purchase_date,item_name,item_price,customer_name,purchase_month
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,2018/01,2019-02-24 01:07:56,商品C,300.0,須賀ひとみ,2019/02
1,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,2018/01,2019-05-08 15:42:01,商品P,1600.0,須賀ひとみ,2019/05
2,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,2018/01,2019-07-03 07:49:05,商品M,1300.0,須賀ひとみ,2019/07
3,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,2018/01,2019-01-02 13:52:15,商品L,1200.0,須賀ひとみ,2019/01
4,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,2018/01,2019-06-29 04:58:36,商品R,1800.0,須賀ひとみ,2019/06


# END